<a href="https://colab.research.google.com/github/howDareYouSayThat/Pytorch_Learning/blob/main/Pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick start

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download data from datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
# Create data loaders.
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
from torch.nn.modules.linear import Linear
from torch.nn.modules.activation import ReLU, Sigmoid

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr = 1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [ ]:
# Train
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    # batch(0-937), batch_size = 64 => batch * batch_size = 938 * 64 = 60032 > 60000
    for batch, (X, y) in enumerate(dataloader): # Transform dataloader into a enumerate
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# Test
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epoch = 5
for t in range(epoch):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("DONE")

Epoch 1
-------------------------------
loss: 2.299803  [    0/60000]
loss: 1.039530  [ 6400/60000]
loss: 0.567578  [12800/60000]
loss: 0.738725  [19200/60000]
loss: 0.608938  [25600/60000]
loss: 0.528378  [32000/60000]
loss: 0.498577  [38400/60000]
loss: 0.671090  [44800/60000]
loss: 0.604464  [51200/60000]
loss: 0.479418  [57600/60000]
Test Error: 
 Accuracy: 82.5%, Avg loss: 0.494904 

Epoch 2
-------------------------------
loss: 0.379146  [    0/60000]
loss: 0.501619  [ 6400/60000]
loss: 0.347376  [12800/60000]
loss: 0.518911  [19200/60000]
loss: 0.447661  [25600/60000]
loss: 0.430252  [32000/60000]
loss: 0.395270  [38400/60000]
loss: 0.608672  [44800/60000]
loss: 0.541102  [51200/60000]
loss: 0.405657  [57600/60000]
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.436584 

Epoch 3
-------------------------------
loss: 0.282753  [    0/60000]
loss: 0.423398  [ 6400/60000]
loss: 0.300400  [12800/60000]
loss: 0.460732  [19200/60000]
loss: 0.375126  [25600/60000]
loss: 0.405521  [32000/600

In [ ]:
# Saving Models
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
# Loading Models
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# Put the model into evaluation mode
model.eval()
x, y = test_data[6][0], test_data[6][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Coat", Actual: "Coat"
